# A study on the global cumulative mass balance of all glaciers and its uncertainty in the dataset.

## Import packages

In [ ]:
import fsspec
import geopandas as gpd
import matplotlib.pyplot as plt
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
year_start = 1975
year_stop = 2020

# Shapefile with countries
shapefile_url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip"

## Define request

In [ ]:
collection_id = "derived-gridded-glacier-mass-change"
request = {
    "variable": "glacier_mass_change",
    "product_version": "wgms_fog_2022_09",
    "format": "zip",
    "hydrological_year": [
        f"{year}_{str(year + 1)[-2:] if year + 1 != 2000 else str(year + 1)[:2]}"
        for year in range(year_start, year_stop + 1)
    ],
}

## Download data

In [ ]:
ds = download.download_and_transform(
    collection_id, request, chunks={"hydrological_year": 1}
)
# Customize some attributes
ds["time"].attrs |= {"long_name": "Time", "units": "yr"}
ds["Glacier"].attrs["long_name"] = "Mass change"
ds["Uncertainty"].attrs["long_name"] = "Mass change error"

## Compute maps and timeseries

In [ ]:
# Maps
mask = ds.notnull().any("time")
ds_maps = ds.sum("time").where(mask)
df_maps = ds_maps.stack(latlon=("latitude", "longitude")).dropna("latlon").to_pandas()
gdf_maps = gpd.GeoDataFrame(
    df_maps,
    geometry=gpd.points_from_xy(df_maps["longitude"], df_maps["latitude"]),
    crs="EPSG:4326",
)

# Timeseries
ds_timeseries = ds.sum(("latitude", "longitude"), keep_attrs=True).compute()
for da in ds_timeseries.values():
    da.attrs["units"] += " yr$^{-1}$"

## Plot maps

In [ ]:
legend_kwds = {"shrink": 0.49, "extend": "both"}
column_kwargs = {
    "Glacier": {
        "cmap": "coolwarm_r",
        "vmin": -5,
        "vmax": 5,
        "legend_kwds": legend_kwds
        | {
            "label": "Cumulative mass change (Gt)",
        },
    },
    "Uncertainty": {
        "cmap": "coolwarm",
        "vmin": 0,
        "vmax": 5,
        "legend_kwds": legend_kwds
        | {
            "label": "Total mass change error (Gt)",
        },
    },
}
for var_name, kwargs in column_kwargs.items():
    with fsspec.open(f"simplecache::{shapefile_url}") as f:
        gdf_countries = gpd.read_file(f)
    ax = gdf_countries.boundary.plot(
        figsize=(20, 16), facecolor="none", edgecolor="black", linewidth=0.25
    )
    gdf_maps.plot(
        ax=ax, c=var_name, markersize=5, column=var_name, legend=True, **kwargs
    )
    ax.axis("off")
    if var_name == "Glacier":
        title = "Figure 1. Cumulative glacier mass change"
    elif var_name == "Uncertainty":
        title = "Figure 2. Sum of glacier mass change error"
    ax.set_title(title, fontsize=25)
    plt.show()

## Plot timeseries

In [ ]:
fig, axs = plt.subplots(2, 1, layout="constrained")
for ax, da in zip(axs, ds_timeseries.values()):
    da.plot(ax=ax)
    ax.grid()
    ax.set_title("Glacier " + da.attrs["long_name"])
_ = fig.suptitle("Figure 3. Total annual glacier mass changes and its uncertainties.")

## Plot timeseries on top of errors

In [ ]:
fig, ax = plt.subplots()
line = ds_timeseries["Glacier"]
error = ds_timeseries["Uncertainty"] / 2
ax.fill_between(line["time"], line - error, line + error, alpha=0.5)
line.plot(ax=ax, color="k")
ax.grid()
_ = ax.set_title("Glacier mass change and its uncertainty")
_ = fig.suptitle("Figure 4. Total annual glacier mass changes and its uncertainty.")